In [22]:
import numpy as np
from utils import *
import torch

# Using new Performance Metrics (Spearmans, Pearsons, etc)

This notebook will walk you through how, using existing results from a TCRP run, calculate performance metrics using other methods. The original paper measures performance via Pearson's correlation, but if you would like to substitute Pearson's for another metric (such as Spearman's), this notebook will be useful!

First, you must enter certain hyperparameters that were used in your TCRP run (these are found in the original `MAML_DRUG.py` run command you executed for this dataset, drug and tissue

In [32]:
num_epochs = 10
num_trials = 20
K = 10

find the folder containing the results for this set of hyperparameters

In [16]:
run_folder = "/results/gCSI_PDTC_predictions/Paclitaxel/PDTC/0.1_0.001_1_12"

# Get Zero Shot Performance

In [43]:
zero_list = []
for i in range(num_epochs):
    zero_prediction = np.load(f"{run_folder}/epochs_{i}/zero_shot_predict.npy")
    zero_label = np.load(f"{run_folder}/epochs_{i}/zero_shot_true.npy")
    zero_list.append(pearson_corr(torch.Tensor(zero_prediction), torch.Tensor(zero_label)))
zero_list = np.array(zero_list)

# Get Few Shot Performance

In [25]:
total_train_list = []
for i in range(num_epochs):
    epoch_list = []
    for k in range(1, K+1):
        sample_list = []
        for j in range(num_trials):
            prediction = np.load(f"{run_folder}/epochs_{i}/{k}_{j}_shot_predict.npy")
            label = np.load(f"{run_folder}/epochs_{i}/{k}_{j}_shot_true.npy")
            sample_list.append(pearson_corr(torch.Tensor(prediction), torch.Tensor(label)))
        epoch_list.append(np.array(sample_list).mean())
    total_train_list.append(epoch_list)

# Find Best Epoch

In order to receive the best performance metrics, the values for the best epoch must be retrieved. The best epoch is written in `log.txt` in the directory `run_folder`

In [35]:
best_epoch = 6

Now, we will subset to obtain zero shot and few shot performance for the best epoch

In [44]:
best_train = total_train_list[best_epoch]
best_zero = zero_list[best_epoch]

# Now, we can visualize the performance metric for this dataset, tissue, drug and epoch (across the number of samples)

In [48]:
print("Zero Shot")
print((best_zero))
print("Few Shot")
best_train

Zero Shot
-0.18085875
Few Shot


[0.19999997615814208,
 0.35000000298023226,
 0.4500000149011612,
 0.5,
 0.5,
 0.6,
 0.6,
 0.5,
 0.5,
 0.6]

We utilized the Pearson's correlation here, but that can easily be replaced in the appopriate places with other performance metrics!